# S2S Competition - Data Processing

## Notebook configuration

In [1]:
BOOT_DASK_CLUSTER: bool = False
    
INPUT_DATA: str = '***BASEDIR***training-input/0.3.0/netcdf'
FILTER_FILE: str = "eccc"

## Imports packages

In [2]:
import dask
import dask.array as da
import dask_jobqueue
import dask_jobqueue

In [29]:
import pathlib
import xarray as xr

import os
import pandas as pd
from tqdm.notebook import tqdm
import time

## Boot Dask Cluster

In [4]:
if BOOT_DASK_CLUSTER:
    cluster = dask_jobqueue.SLURMCluster(
        cores=12,
        processes=6,
        memory='128G',
        env_extra=['source ***HOME***.bash_profile','conda activate s2s'],
        name='agri-dask',
        local_directory='***CACHE***', # METTRE VOTRE LOGIN CRIM ICI
        walltime='3:00:00'
    )

# Extract informations from Data

In [5]:
input_path = pathlib.Path(INPUT_DATA)

In [6]:
eccc_files = sorted([x for x in input_path.iterdir() if x.stem.startswith(FILTER_FILE)])

In [59]:
variables: set = set([os.path.splitext(os.path.basename(str(file)))[0].split("-")[-2] for file in eccc_files])    

In [97]:
eccc_files_df: pd.DataFrame = pd.DataFrame(columns=["File", "Variable_shortname", "Variable_longname", "Variable_cfname", "Unit", "Step_type", "Level_type", "Level", "Number_of_points", "Missing_values", "Dimensions"])

for file, i in zip(eccc_files, tqdm(range(0, len(eccc_files)), desc ="Extract informations from dataset files")):
    ds = xr.open_dataset(file)
    var_name = list(ds.data_vars.keys())
    filename = os.path.splitext(os.path.basename(str(file)))[0]
    var_shortname = ds[var_name[0]].GRIB_shortName
    long_name = ds[var_name[0]].long_name
    var_unit = ds[var_name[0]].GRIB_units  
    var_cfname = ds[var_name[0]].GRIB_cfVarName
    missing_values = ds[var_name[0]].GRIB_missingValue 
    level = ds[var_name[0]].realization.data
    level_type = ds[var_name[0]].GRIB_typeOfLevel
    number_of_points = ds[var_name[0]].GRIB_numberOfPoints
    step_type = ds[var_name[0]].GRIB_stepType 
    dim = set(["forecast_time", "latitude", "lead_time", "longitude", "realization", "valid_time"]).symmetric_difference(set(list(ds[var_name[0]].coords.keys())))
    
    eccc_files_df = eccc_files_df.append({"File": filename, 
                                          "Variable_shortname": var_shortname,  
                                          "Variable_longname": long_name, 
                                          "Variable_cfname": var_cfname, 
                                          "Unit": var_unit,
                                          "Step_type": step_type,
                                          "Level_type": level_type,
                                          "Level": level,
                                          "Number_of_points": number_of_points,
                                          "Missing_values": missing_values,
                                          "Dimensions": dim}, 
                                         ignore_index=True)

Extract informations from dataset files:   0%|          | 0/901 [00:00<?, ?it/s]

## Check available variables with some informations

In [99]:
sub_eccc_files_df = eccc_files_df.drop_duplicates(subset=['Variable_shortname'])
sub_eccc_files_df.set_index("Variable_shortname", inplace=True)
sub_eccc_files_df

,File,Variable_longname,Variable_cfname,Unit,Step_type,Level_type,Level,Number_of_points,Missing_values,Dimensions
Variable_shortname,,,,,,,,,,
ci,eccc-hindcast-ci-20200102,Sea ice area fraction,siconc,(0 - 1),avg,surface,"[0, 1, 2, 3]",29040,9999,{}
gh,eccc-hindcast-gh-20200102,Geopotential Height,gh,gpm,instant,isobaricInhPa,"[0, 1, 2, 3]",29040,9999,{plev}
lsm,eccc-hindcast-lsm-20200102,Land-sea mask,lsm,(0 - 1),instant,surface,[0],29040,9999,{}
msl,eccc-hindcast-msl-20200102,Mean sea level pressure,msl,Pa,instant,meanSea,"[0, 1, 2, 3]",29040,9999,{meanSea}
q,eccc-hindcast-q-20200102,Specific humidity,q,kg kg**-1,instant,isobaricInhPa,"[0, 1, 2, 3]",29040,9999,{plev}
rsn,eccc-hindcast-rsn-20200102,Snow density,rsn,kg m**-3,avg,surface,"[0, 1, 2, 3]",29040,9999,{}
sp,eccc-hindcast-sp-20200102,Surface pressure,sp,Pa,instant,surface,"[0, 1, 2, 3]",29040,9999,{}
sst,eccc-hindcast-sst-20200102,Sea surface temperature,sst,K,avg,surface,"[0, 1, 2, 3]",29040,9999,{}
t,eccc-hindcast-t-20200102,Temperature,t,K,instant,isobaricInhPa,"[0, 1, 2, 3]",29040,9999,{plev}


In [91]:
ds = xr.open_dataset(eccc_files[0])
ds

<xarray.Dataset>
Dimensions:        (forecast_time: 20, latitude: 121, lead_time: 32, longitude: 240, realization: 4)
Coordinates:
  * realization    (realization) int64 0 1 2 3
  * forecast_time  (forecast_time) datetime64[ns] 1998-01-02 ... 2017-01-02
  * lead_time      (lead_time) timedelta64[ns] 1 days 2 days ... 31 days 32 days
  * latitude       (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
  * longitude      (longitude) float64 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
    valid_time     (forecast_time, lead_time) datetime64[ns] ...
Data variables:
    siconc         (realization, forecast_time, lead_time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal 
    history:                 2021-05-11T09:30 GRIB to CDM+CF via cfgrib-0.9.9...